# 10주차 실습: LLMOps 로그 수집 & 간단 분석

이 노트북은 **9주차 FastAPI/추론 실습**을 이어서, 10주차에 배운 **LLMOps(로그 수집·관찰)**을 실습하기 위한 템플릿입니다.

👉 목표:
- LLM을 여러 번 호출하고,
- 각 호출을 공통 스키마로 **CSV(or Langfuse)**에 기록하고,
- 간단한 통계를 뽑아보는 것까지 진행합니다.

※ 이 노트북은 **직접 실행용 템플릿**입니다. OpenAI 키, Langfuse 키 등은 각각 환경에서 세팅해주세요.

## 1. 환경 준비

- Python 3.10+
- 패키지: `openai`, `pandas`, `python-dotenv`(선택), `langfuse`(선택)

터미널에서 다음과 같이 설치할 수 있습니다:

```bash
pip install openai pandas python-dotenv langfuse
```

또는 9주차에 사용한 가상환경/conda 환경을 그대로 사용해도 됩니다.

In [15]:
# (선택) 필요한 패키지 설치 셀 – 이미 설치되어 있다면 실행할 필요 없습니다.
!pip install -q openai pandas python-dotenv langfuse


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from pathlib import Path


## 2. 설정 & 공통 상수

9주차에서 사용한 환경과 이어서 사용합니다.

- OpenAI API 키는 환경변수 `OPENAI_API_KEY`로 설정했다고 가정합니다.
- (선택) Langfuse를 사용할 경우 `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_HOST`를 설정합니다.

로컬 개발 환경에서는 `.env` 파일에 다음과 같이 적어둘 수 있습니다:

```env
OPENAI_API_KEY=sk-...
LANGFUSE_PUBLIC_KEY=...
LANGFUSE_SECRET_KEY=...
LANGFUSE_HOST=http://localhost:3000
```

In [16]:
import os
import time
from datetime import datetime

import pandas as pd

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    # python-dotenv가 없어도 동작은 가능
    pass

# OpenAI 설정 (새 Python SDK 기준)
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    print("⚠️ OPENAI_API_KEY 환경변수가 설정되어 있지 않습니다. 이 노트북의 LLM 호출 부분은 실행되지 않을 수 있습니다.")

client = OpenAI(api_key=OPENAI_API_KEY)

# 로그 디렉토리 설정
LOG_DIR = Path("logs")
LOG_DIR.mkdir(exist_ok=True)
CSV_LOG_PATH = LOG_DIR / "llm_responses.csv"

## 3. LLM 호출 함수 정의

9주차 실습에서 사용했던 LLM 호출 코드를 함수로 정리합니다.

- 입력: `prompt`, `model`, `temperature`, `max_tokens`
- 출력: (response_text, latency_ms, usage_dict)

여기서는 OpenAI Chat Completions API를 예시로 사용합니다. (실제 환경에 맞게 수정 가능)

In [17]:
def call_llm(prompt: str,
             model: str = "gpt-4o-mini",
             temperature: float = 0.2,
             max_tokens: int = 512):
    """LLM을 한 번 호출하고, 응답 텍스트/지연시간/토큰 사용량을 반환합니다."""
    if OPENAI_API_KEY is None:
        raise RuntimeError("OPENAI_API_KEY가 설정되어 있지 않아 LLM을 호출할 수 없습니다.")

    start = time.time()
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant for an LLMOps lecture demo."},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    end = time.time()

    latency_ms = int((end - start) * 1000)

    message = completion.choices[0].message
    text = message.content

    usage = completion.usage
    usage_dict = {
        "input_tokens": usage.prompt_tokens if usage else None,
        "output_tokens": usage.completion_tokens if usage else None,
        "total_tokens": usage.total_tokens if usage else None,
    }

    return text, latency_ms, usage_dict

## 4. CSV 기반 Logger 구현

Langfuse를 쓰지 못하는 환경에서도 실습할 수 있도록, **CSV 기반 LLMOps 로거**를 먼저 구현합니다.

- 각 호출을 한 줄(row)로 저장합니다.
- 스키마는 10주차 강의에서 정의한 공통 스키마의 축소판을 사용합니다.

In [18]:
import uuid

CSV_COLUMNS = [
    "id", "timestamp", "endpoint", "user_id", "session_id",
    "prompt", "prompt_version", "model",
    "latency_ms", "input_tokens", "output_tokens", "total_tokens",
    "cost", "status_code", "feedback",
]


class CsvLogger:
    def __init__(self, csv_path: Path, model_price_per_1k_tokens: float = 0.0005):
        self.csv_path = csv_path
        self.price_per_1k_tokens = model_price_per_1k_tokens

        if not self.csv_path.exists():
            df = pd.DataFrame(columns=CSV_COLUMNS)
            df.to_csv(self.csv_path, index=False)

    def _estimate_cost(self, total_tokens: int | None) -> float | None:
        if total_tokens is None:
            return None
        return round(self.price_per_1k_tokens * (total_tokens / 1000.0), 6)

    def log(self,
            prompt: str,
            prompt_version: str,
            model: str,
            latency_ms: int,
            usage: dict | None = None,
            endpoint: str = "/llmops/demo",
            user_id: str = "demo_user",
            session_id: str | None = None,
            status_code: int = 200,
            feedback: int | None = None):

        if session_id is None:
            session_id = str(uuid.uuid4())

        usage = usage or {}
        input_tokens = usage.get("input_tokens")
        output_tokens = usage.get("output_tokens")
        total_tokens = usage.get("total_tokens")

        cost = self._estimate_cost(total_tokens)

        row = {
            "id": str(uuid.uuid4()),
            "timestamp": datetime.utcnow().isoformat(),
            "endpoint": endpoint,
            "user_id": user_id,
            "session_id": session_id,
            "prompt": prompt,
            "prompt_version": prompt_version,
            "model": model,
            "latency_ms": latency_ms,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": total_tokens,
            "cost": cost,
            "status_code": status_code,
            "feedback": feedback,
        }

        df = pd.DataFrame([row])
        df.to_csv(self.csv_path, mode="a", header=False, index=False)

        return row


csv_logger = CsvLogger(CSV_LOG_PATH)
print(f"✅ CSV Logger 준비 완료: {CSV_LOG_PATH}")

✅ CSV Logger 준비 완료: logs/llm_responses.csv


## 5. (선택) Langfuse Logger 구현

Langfuse 서버(클라우드 또는 로컬)가 있고, `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_HOST`를 설정했다면
아래 코드를 사용해 **Langfuse trace**도 동시에 남길 수 있습니다.

설정이 없으면 이 셀은 경고만 출력하고 넘어갑니다.

In [19]:
# === (수정 버전) Langfuse 연동 ===
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")

# Langfuse v3에서는 기본적으로 LANGFUSE_BASE_URL 을 사용합니다.
LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL") or os.getenv("LANGFUSE_HOST")

langfuse_client = None

if LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY:
    try:
        # v3 기준: 직접 Langfuse(...) 대신 get_client() 사용
        from langfuse import get_client

        langfuse_client = get_client()
        print("✅ Langfuse 클라이언트 초기화 완료 (v3 스타일)")
    except Exception as e:
        print("⚠️ Langfuse 초기화 실패:", e)
        langfuse_client = None
else:
    print("ℹ️ Langfuse 환경변수가 설정되지 않았습니다. CSV 로깅만 사용합니다.")


def log_to_langfuse(
    prompt: str,
    response: str,
    prompt_version: str,
    model: str,
    latency_ms: int,
    usage: dict | None = None,
    user_id: str = "demo_user",
):
    """
    Langfuse가 설정된 경우, 하나의 span(=관찰)을 남깁니다.
    v3 SDK에서는 client.trace(...)가 아니라 start_as_current_span(...)을 사용합니다.
    """
    if langfuse_client is None:
        return None

    usage = usage or {}
    try:
        # 간단하게 span 하나를 만들고, input/output + 메타데이터를 남김
        with langfuse_client.start_as_current_span(
            name="week10-llmops-demo",
            input={
                "prompt": prompt,
                "prompt_version": prompt_version,
                "model": model,
            },
        ) as span:
            # span 자체에 output/메타데이터 기록
            span.update(
                output={"response": response},
                metadata={
                    "prompt_version": prompt_version,
                    "model": model,
                    "latency_ms": latency_ms,
                    "input_tokens": usage.get("input_tokens"),
                    "output_tokens": usage.get("output_tokens"),
                    "total_tokens": usage.get("total_tokens"),
                },
            )

            # trace 레벨 속성도 업데이트 가능 (user_id 등)
            span.update_trace(
                user_id=user_id,
                tags=["week10-llmops-demo"],
            )

        return True
    except Exception as e:
        # Langfuse만 실패해도 전체 실습이 깨지지 않도록 그냥 경고만 출력
        print("⚠️ Langfuse 로깅 실패:", e)
        return None


✅ Langfuse 클라이언트 초기화 완료 (v3 스타일)


## 6. 프롬프트 버전별로 여러 번 호출해보기

이제 실제로 **프롬프트 버전 A/B**를 정의하고, 각각 여러 번 호출하면서 로그를 남겨봅니다.

- `PROMPT_V1`, `PROMPT_V2` 두 가지를 준비합니다.
- 각 버전당 3회 이상 호출해보는 것을 권장합니다.
- Langfuse가 설정되어 있으면 Langfuse + CSV, 아니면 CSV만 기록됩니다.

In [20]:
PROMPT_V1 = "LLMOps를 처음 배우는 대학생에게, 아주 간단하게 한 문단으로 설명해줘."
PROMPT_V2 = "당신은 머신러닝 운영 전문가입니다. MLOps와 LLMOps의 차이를 핵심 키워드 위주로 3줄 이내로 정리해줘."

def run_batch_calls(prompts: list[tuple[str, str]],
                    model: str = "gpt-4o-mini",
                    n_calls_per_prompt: int = 3):
    """(prompt_text, prompt_version) 목록을 받아 각 버전마다 여러 번 호출합니다."""
    rows = []
    for prompt_text, prompt_version in prompts:
        print(f"\n=== Prompt version: {prompt_version} ===")
        for i in range(n_calls_per_prompt):
            print(f"  - call {i+1}/{n_calls_per_prompt}...", end=" ")
            try:
                response, latency_ms, usage = call_llm(prompt_text, model=model)
                row = csv_logger.log(
                    prompt=prompt_text,
                    prompt_version=prompt_version,
                    model=model,
                    latency_ms=latency_ms,
                    usage=usage,
                )
                log_to_langfuse(
                    prompt=prompt_text,
                    response=response,
                    prompt_version=prompt_version,
                    model=model,
                    latency_ms=latency_ms,
                    usage=usage,
                )
                print(f"✅ {latency_ms} ms, tokens={usage.get('total_tokens') if usage else None}")
                rows.append(row)
            except Exception as e:
                print("❌ 호출 실패:", e)
    return rows

# 실습 때 직접 실행하세요 (API 키 필요)
_ = run_batch_calls([
    (PROMPT_V1, "v1"),
    (PROMPT_V2, "v2"),
], model="gpt-4o-mini", n_calls_per_prompt=3)

print("준비 완료 ✅\n실제 LLM 호출은 위 run_batch_calls(...) 주석을 해제하고 실행해보세요.")


=== Prompt version: v1 ===
  - call 1/3... ✅ 3113 ms, tokens=152
  - call 2/3... ✅ 2891 ms, tokens=150
  - call 3/3... ✅ 3092 ms, tokens=165

=== Prompt version: v2 ===
  - call 1/3... ✅ 4237 ms, tokens=186
  - call 2/3... ✅ 2881 ms, tokens=195
  - call 3/3... ✅ 3452 ms, tokens=177
준비 완료 ✅
실제 LLM 호출은 위 run_batch_calls(...) 주석을 해제하고 실행해보세요.


## 7. CSV 로그 불러와서 간단 분석하기

이제 `logs/llm_responses.csv` 파일을 불러와서 간단한 통계를 계산해 봅니다.

예시:
- 프롬프트 버전별 평균 지연시간
- 프롬프트 버전별 평균 토큰 수
- 모델별 평균 비용

※ LLM 호출을 실제로 실행한 뒤에 이 셀을 돌려야 의미있는 값이 나옵니다.

In [21]:
if CSV_LOG_PATH.exists():
    df = pd.read_csv(CSV_LOG_PATH)
    print("로그 레코드 수:", len(df))
    display(df.tail())
else:
    print("아직 CSV 로그 파일이 없습니다. 먼저 LLM 호출을 실행하고 다시 시도하세요.")

로그 레코드 수: 18


,id,timestamp,endpoint,user_id,session_id,prompt,prompt_version,model,latency_ms,input_tokens,output_tokens,total_tokens,cost,status_code,feedback
13,79abd102-9cb7-49ad-ace7-522501051a83,2025-11-03T11:28:14.943613,/llmops/demo,demo_user,cd70670e-f0fb-45cf-bcbe-84b41aca020d,"LLMOps를 처음 배우는 대학생에게, 아주 간단하게 한 문단으로 설명해줘.",v1,gpt-4o-mini,2891,47,103,150,0.000075,200,NaN
14,3213bc85-2f64-43f9-b6ca-ff18ac1f35fd,2025-11-03T11:28:18.037747,/llmops/demo,demo_user,c5b7fe29-e2ef-49aa-b8ac-8f0a7ea6322f,"LLMOps를 처음 배우는 대학생에게, 아주 간단하게 한 문단으로 설명해줘.",v1,gpt-4o-mini,3092,47,118,165,0.000082,200,NaN
15,fbf4dc33-0838-4036-bee9-3611f3439506,2025-11-03T11:28:22.276691,/llmops/demo,demo_user,6ed09225-7792-40f4-9b38-26a1b4e12344,당신은 머신러닝 운영 전문가입니다. MLOps와 LLMOps의 차이를 핵심 키워드 ...,v2,gpt-4o-mini,4237,63,123,186,0.000093,200,NaN
16,6b345f8b-0066-4c61-af18-58e06f9ddfe1,2025-11-03T11:28:25.160570,/llmops/demo,demo_user,b20f3c68-3b8e-4e68-964b-4e9336b397c1,당신은 머신러닝 운영 전문가입니다. MLOps와 LLMOps의 차이를 핵심 키워드 ...,v2,gpt-4o-mini,2881,63,132,195,0.000098,200,NaN
17,c14b3be2-fb13-4c07-93ce-e6f83369d635,2025-11-03T11:28:28.617761,/llmops/demo,demo_user,7138d3eb-d8c0-47b9-9c6f-4bac3939b32e,당신은 머신러닝 운영 전문가입니다. MLOps와 LLMOps의 차이를 핵심 키워드 ...,v2,gpt-4o-mini,3452,63,114,177,0.000088,200,NaN


In [22]:
if CSV_LOG_PATH.exists():
    df = pd.read_csv(CSV_LOG_PATH)

    print("\n📊 프롬프트 버전별 평균 latency_ms")
    if "prompt_version" in df.columns and "latency_ms" in df.columns:
        display(df.groupby("prompt_version")["latency_ms"].mean())
    else:
        print("필요한 컬럼이 없습니다.")

    print("\n📊 프롬프트 버전별 평균 total_tokens")
    if "prompt_version" in df.columns and "total_tokens" in df.columns:
        display(df.groupby("prompt_version")["total_tokens"].mean())
    else:
        print("필요한 컬럼이 없습니다.")

    print("\n📊 모델별 평균 cost")
    if "model" in df.columns and "cost" in df.columns:
        display(df.groupby("model")["cost"].mean())
    else:
        print("필요한 컬럼이 없습니다.")


📊 프롬프트 버전별 평균 latency_ms


prompt_version
v1    3250.555556
v2    3743.888889
Name: latency_ms, dtype: float64


📊 프롬프트 버전별 평균 total_tokens


prompt_version
v1    157.222222
v2    188.000000
Name: total_tokens, dtype: float64


📊 모델별 평균 cost


model
gpt-4o-mini    0.000086
Name: cost, dtype: float64

## 8. 리포트 템플릿 (과제용)

아래 텍스트 셀을 복사해서 `report.md` 또는 과제 제출용 문서에 붙여넣고, 자신의 결과를 채워보세요.

```markdown
# 10주차 LLMOps 실습 리포트

## 1. 실험 설정
- 사용 모델: gpt-4o-mini
- 프롬프트 버전:
  - v1: ...
  - v2: ...
- 각 버전당 호출 횟수: 3회

## 2. 결과 요약
- v1 평균 latency: XXX ms
- v2 평균 latency: YYY ms
- v1 평균 total_tokens: AAA
- v2 평균 total_tokens: BBB
- v1/v2 비용 차이: ...

## 3. 인사이트
- v2는 답변이 더 길어서 토큰/비용이 늘어났지만, 설명이 더 친절했다.
- v1은 짧고 빠르지만, LLMOps를 처음 배우는 사람 입장에서는 이해가 어렵다는 피드백이 있었다.
- 운영 관점에서는 v2 프롬프트를 사용하되, 토큰을 줄이기 위한 개선이 필요해 보인다.
```

## 9. 마무리

- 이 노트북을 통해 **LLM 호출 → 로그 남기기 → 통계 보기**까지 한 번의 사이클을 경험했습니다.
- 11주차에서는 이 로그/응답들을 활용해 **평가(Evaluation)**와 **RAG 품질 측정**을 진행하게 됩니다.

필요하다면 이 노트북을 복사해서 자신의 프로젝트/과제에 맞게 **프롬프트, 모델, 로그 스키마**를 수정해보세요. 🚀